In [1]:
!pip install dgl
from os import listdir
import dgl
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

     |████████████████████████████████| 3.0MB 8.6MB/s 


DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [3]:
filename = "/content/drive/My Drive/adaptec1.nets"

edge_file = "/content/add_edges.py"
edge_file_handle = open(edge_file, "w+")

node_count = 0

with open(filename,encoding="utf8") as f:
    netSection = False
    edges = 0
    lineCount = 0
    src=[]
    dest=[]
    for line in f:
        lineCount += 1
        #Newline
        if len(line) < 2:
            continue
            
        # Comments
        if line[0] == '#':
            continue
        #print(line)
        splitLine = line.split()
        #print("Words = ", len(splitLine))
        if(splitLine[0] == 'NetDegree'):
            
            if edges > 0: #Previous NetDegree not complete!
                print("Missed an edge at line: ", lineCount-1)
                
            edges = int(splitLine[2])
            netSection = True
            src=[]
            dest=[]
            #G.add_nodes(num)
            
        elif netSection == True:
            node_name = splitLine[0][1:]
            node_val  = int(node_name)
            if node_count < node_val+1:
                node_count = node_val+1
                
            if(splitLine[1] == 'O'):
                src.append(node_name)
                edges -= 1
            elif(splitLine[1] == 'I'):
                dest.append(node_name)
                edges -= 1
        
        if netSection == True:
            if edges == 0:
                if (len(src)<1):
                    print("No source edge, skipping")
                    continue
                if (len(dest)<1):
                    print("No dest edge, skipping")
                    continue
                    
                netSection = False
                for des_idx in range(len(dest)):
                    cmd = "G.add_edges("+src[0]+","+dest[des_idx]+")"+"\n"
                    edge_file_handle.write(cmd)
                    
        #if lineCount > 100:
        #    break


edge_file_handle.close()


No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No source edge, skipping
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584552
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584660
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584723
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584753
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584781
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  584978
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  585078
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  585138
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  585199
No dest edge, skipping
No dest edge, skipping
Missed an edge at line:  585306
No dest edge, skipping
No dest edge, skipping
Missed an edge at line: 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [28]:
G = dgl.DGLGraph()
G.add_nodes(node_count)
exec(open(edge_file).read())
print("Nodes = ", G.number_of_nodes())
print("Edges = ", G.number_of_edges())

Nodes =  211447
Edges =  720134


In [0]:
nx_G = G.to_networkx().to_undirected()

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
embed = nn.Embedding(G.number_of_nodes(),6)

In [0]:
G.ndata['feat'] = embed.weight

In [0]:
from dgl.nn.pytorch import GraphConv

In [0]:
class GCN(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(GCN,self).__init__()
        self.conv1 = GraphConv(input_size,hidden_size)
        self.conv2 = GraphConv(hidden_size,num_classes)
    
    def forward(self,G,inputs):
        h = self.conv1(G,inputs)
        h = torch.relu(h) 
        h = self.conv2(G,h)
        return h

In [0]:
net = GCN(6,5,20)

In [0]:
inputs = embed.weight
a = np.linspace(0,G.number_of_nodes()-1,20,dtype=int)
labelled_nodes = torch.tensor(a)
labels = torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])

# Training Model

In [37]:
import itertools
import time

optimizer = torch.optim.Adam(itertools.chain(net.parameters(), embed.parameters()), lr=0.01)
all_logits = []

start_time = time.time()

for epoch in range(150):
    logits = net(G, inputs)
    all_logits.append(logits.detach())
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[labelled_nodes], labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print('Epoch {} | Loss: {}'.format(epoch + 1, loss.item()))

end_time = time.time()
print("Training Time: {}".format(end_time - start_time))

Epoch 1 | Loss: 3.0929489135742188
Epoch 2 | Loss: 3.064601182937622
Epoch 3 | Loss: 3.037524700164795
Epoch 4 | Loss: 3.011530876159668
Epoch 5 | Loss: 2.9867963790893555
Epoch 6 | Loss: 2.9630579948425293
Epoch 7 | Loss: 2.9399654865264893
Epoch 8 | Loss: 2.917433261871338
Epoch 9 | Loss: 2.895404815673828
Epoch 10 | Loss: 2.8735756874084473
Epoch 11 | Loss: 2.85172700881958
Epoch 12 | Loss: 2.829862117767334
Epoch 13 | Loss: 2.8077759742736816
Epoch 14 | Loss: 2.7854065895080566
Epoch 15 | Loss: 2.762728214263916
Epoch 16 | Loss: 2.7398252487182617
Epoch 17 | Loss: 2.717055320739746
Epoch 18 | Loss: 2.693718433380127
Epoch 19 | Loss: 2.6697216033935547
Epoch 20 | Loss: 2.645113945007324
Epoch 21 | Loss: 2.6200108528137207
Epoch 22 | Loss: 2.594428300857544
Epoch 23 | Loss: 2.568704128265381
Epoch 24 | Loss: 2.542356252670288
Epoch 25 | Loss: 2.515340566635132
Epoch 26 | Loss: 2.4876489639282227
Epoch 27 | Loss: 2.4597136974334717
Epoch 28 | Loss: 2.4311728477478027
Epoch 29 | Loss: 

In [0]:
pos = {}
cluster_nodes = {} # Every value in key->value pair includes the cluster centre node as well.
for v in range(G.number_of_nodes()):
  pos[v] = all_logits[99][v].numpy()
  cls = pos[v].argmax()
  if cluster_nodes.get(cls) :
    cluster_nodes[cls].append(v)
  else:
    cluster_nodes[cls] = [] 
    cluster_nodes[cls].append(v)

In [61]:
print(cluster_nodes)

{0: [0, 997, 1246, 1272, 1610, 1691, 1972, 1974, 2107, 2185, 2186, 2273, 2431, 2565, 2885, 2946, 2972, 3167, 3444, 3445, 3598, 3856, 4189, 4198, 4413, 4426, 4470, 4475, 4492, 4594, 4726, 4734, 4792, 4939, 5020, 5182, 5233, 5282, 5326, 5327, 5331, 5351, 5386, 5399, 5405, 5406, 5455, 5465, 5485, 5492, 5502, 5567, 5597, 5605, 5637, 5655, 5658, 5685, 5688, 5708, 5729, 5734, 5736, 5818, 5832, 5861, 5870, 5938, 5939, 5941, 6004, 6005, 6073, 6159, 6164, 6339, 6559, 6580, 6582, 6779, 6788, 6791, 7079, 7089, 7251, 7267, 7304, 7367, 7529, 7560, 7611, 7818, 7915, 8032, 8067, 8302, 8549, 9003, 9088, 9094, 9129, 9132, 9285, 9297, 9558, 9636, 9970, 10387, 10475, 10512, 10522, 10577, 10616, 10628, 10675, 10679, 10683, 10856, 10893, 10965, 10986, 11064, 11141, 11162, 11210, 11271, 11340, 11399, 11403, 11412, 11415, 11546, 11553, 11610, 11621, 11667, 11681, 11698, 11700, 11706, 11751, 11761, 11773, 11866, 11973, 11974, 12017, 12196, 12259, 12339, 12378, 12385, 12415, 12417, 12637, 12721, 12845, 12980, 

In [0]:
number_dict = {} # Dictionary mapping the node label to the number of nodes in a particular cluster
for key,value in cluster_nodes.items():
  number_dict[key] = len(value)

number_dict = sorted(number_dict.items()) # Sorting dictionary by keys in increasing order

In [41]:
print(number_dict)

[(0, 2857), (1, 14810), (2, 9884), (3, 3658), (4, 5652), (5, 60), (6, 1714), (7, 6496), (8, 12221), (9, 6037), (10, 65518), (11, 5822), (12, 2877), (13, 12830), (14, 13420), (15, 18527), (16, 3811), (17, 14721), (18, 6183), (19, 4349)]


# Clustering for 0th Node Cluster

In [0]:
import re

In [90]:
 a = "G.add_edges(197110,0)"
 print(re.split(r'[(,)]',a)) # Split using regular expressions

['G.add_edges', '197110', '0', '']


In [0]:
splitList = []
with open("/content/add_edges.py", encoding="utf8") as f:
  for line in f:
     splitLine = re.split(r'[(,)]',line)
     splitList.append(splitLine)


In [0]:
src = []
dst = []
for ix in cluster_nodes[0]:
  for val in splitList:
    if(int(val[1]) == ix):
      src.append(int(val[1]))
      dst.append(int(val[2]))

src = np.array(src)
dst = np.array(dst)

In [104]:
print(src[:10])

[   0  997  997  997 1246 1246 1272 1272 1272 1272]


In [105]:
print(dst[:10])

[    23  60962  45306  45275  61337  36935 104071 104001 103999 103996]


In [106]:
src.shape,dst.shape

((6066,), (6066,))

In [0]:
def build_graph():
  u = np.concatenate([src,dst])
  v = np.concatenate([dst,src])

  return dgl.DGLGraph((u,v))

In [109]:
G1 = build_graph()
print("Nodes = {}".format(G1.number_of_nodes()))
print("Edges = {}".format(G1.number_of_edges()))

Nodes = 211443
Edges = 12132


In [0]:
nx_G = G1.to_networkx().to_undirected()

In [0]:
pos1 = nx.kamada_kawai_layout(nx_G1)